In [89]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import time
import re

# Лишняя фигня, тыкает по порядку на цифры

""" n = 2

while n > 0:
    browser.find_element_by_link_text('{}'.format(n)).click()
    n = n + 1 """


base_url = 'https://old.cbr.ru/press/month_archive/?'
page_part = 'page='

all_urls = []


for i in range(2, 169):
    url_gen = base_url + page_part + str(i)
    r = requests.get(url_gen)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    press_urls = soup.find(
        'table', class_='text_and_dates').find_all('a')
    for element in press_urls:
        press_url = element.get('href')
        all_urls.append(press_url)

In [145]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

data = []

for url in all_urls[:21]:

    try:
        r = requests.get(url)

        html = r.text

        soup = BeautifulSoup(html, 'lxml')

        date = soup.find(
            'div', class_='col-md-6 col-12 news-info-line_date').get_text()

        try:
            lead_text = soup.find('div', class_='lead-text').get_text()
        except:
            pass

        landing_text = soup.find('div', class_='landing-text').get_text()

        try:

            caption = soup.find('div', class_='caption').get_text()
            landing_text = landing_text.replace(caption, '')

        except:
            pass

    except:
        continue

    data.append([date, landing_text])


df = pd.DataFrame(data, columns=['date', 'landing_text'])

In [146]:
df

,date,landing_text
0,31 января 2020 года,\n\nБанк России приказом от 31.01.2020 № ОД-16...
1,30 января 2020 года,\nБанк России принял решение начиная с 2022 го...
2,30 января 2020 года,\n\nВышел «Вестник Банка России» № 11–12 (2147...
3,29 января 2020 года,\n\nПриказами Банка России от 05.09.2019 № ОД-...
4,29 января 2020 года,\n\nВышел «Вестник Банка России» № 10 (2146) о...
5,29 января 2020 года,"\nБанк России принял решение о реализации мер,..."
6,29 января 2020 года,\n\nВ соответствии с решением Совета директоро...
7,28 января 2020 года,\n\nБанк России пригласил инвестиционных консу...
8,27 января 2020 года,\n\nВышел «Вестник Банка России» № 9 (2145) от...
9,24 января 2020 года,\n\nВышел «Вестник Банка России» № 8 (2144) от...


In [147]:
def is_there_number(string):
    return any(i.isdigit() for i in string)

In [148]:
def is_not_blank(s):
    return bool(s and s.strip())

In [149]:
from alphabet_detector import AlphabetDetector
import nltk
from nltk.corpus import stopwords
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import matplotlib.pyplot as plt
%matplotlib inline

ad = AlphabetDetector()

In [150]:
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['это', 'год', 'который', 'ао', 'пао', 'также', 'г', 'всё', 'акб', 'кб', 'очень', 'при', 'далее'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [151]:
words_column = []

for text in df.landing_text:
    
    words = text.split()
    
    punct = "!#$%&'()*+, -./:<= >?@[\] ^ _`{ | }~«»№—–"

    table = str.maketrans('', '', punct)

    words_stripped = [w.translate(table) for w in words]
    
    words_stripped1 = [word.replace('При','') if word.endswith('При') else word for word in words_stripped]
    
    #newlist = [x if x != "banana" else "orange" for x in fruits]
    
    normal_words = [morph.parse("{}".format(word))[0].normal_form for word in words_stripped1 if word.isupper() != True]
    
    clean_words = []
    
    for word in normal_words:
        if ad.only_alphabet_chars(u"{}".format(word), "CYRILLIC") and is_there_number(word) == False \
            and is_not_blank(word) and word not in russian_stopwords:
            clean_words.append(word)
        
    words_column.append(clean_words)

df2 = df.assign(words = [word_column for word_column in words_column])   


In [152]:
df2

,date,landing_text,words
0,31 января 2020 года,\n\nБанк России приказом от 31.01.2020 № ОД-16...,"[банк, россия, приказ, отозвать, лицензия, осу..."
1,30 января 2020 года,\nБанк России принял решение начиная с 2022 го...,"[банк, россия, принять, решение, начинать, исп..."
2,30 января 2020 года,\n\nВышел «Вестник Банка России» № 11–12 (2147...,"[выйти, вестник, банка, россия, январь, номер,..."
3,29 января 2020 года,\n\nПриказами Банка России от 05.09.2019 № ОД-...,"[приказ, банка, россия, общество, отозвать, ли..."
4,29 января 2020 года,\n\nВышел «Вестник Банка России» № 10 (2146) о...,"[выйти, вестник, банка, россия, январь, рубрик..."
5,29 января 2020 года,"\nБанк России принял решение о реализации мер,...","[банк, россия, принять, решение, реализация, м..."
6,29 января 2020 года,\n\nВ соответствии с решением Совета директоро...,"[соответствие, решение, совет, директор, банка..."
7,28 января 2020 года,\n\nБанк России пригласил инвестиционных консу...,"[банк, россия, пригласить, инвестиционный, кон..."
8,27 января 2020 года,\n\nВышел «Вестник Банка России» № 9 (2145) от...,"[выйти, вестник, банка, россия, январь, рубрик..."
9,24 января 2020 года,\n\nВышел «Вестник Банка России» № 8 (2144) от...,"[выйти, вестник, банка, россия, январь, номер,..."


Анализ тональности

In [130]:
emo_dict = pd.read_csv('C:/Users/asker/Desktop/emo_dict.txt', sep = ';')

In [131]:
emo_dict

,term,tag,value,pstv,neut,ngtv,dunno,distortion
0,аббат,NEUT,0.3667,0.2574,0.3762,0.0693,0.2970,0.0880
1,аббревиатура,NEUT,0.0000,0.1600,0.7200,0.0000,0.1200,0.0000
2,абзац,NEUT,0.0000,0.1481,0.7037,0.0000,0.1481,0.0000
3,абонемент,NEUT,0.1757,0.2381,0.5810,0.0476,0.1333,0.0571
4,абонентный,NEUT,0.0000,0.0000,0.7200,0.0000,0.2800,0.0000
...,...,...,...,...,...,...,...,...
28192,ёмкий,NEUT,0.4844,0.4038,0.4231,0.0192,0.1538,0.0201
28193,ёмкость,NEUT,0.0769,0.2000,0.6400,0.0000,0.1600,0.0000
28194,ёрзать,NGTV,-0.5826,0.0400,0.3700,0.4700,0.1200,0.0434
28195,ёрш,NEUT,0.1571,0.2233,0.5728,0.0777,0.1262,0.1047


In [132]:
dict_terms = emo_dict.term.tolist()

In [153]:
df2

,date,landing_text,words
0,31 января 2020 года,\n\nБанк России приказом от 31.01.2020 № ОД-16...,"[банк, россия, приказ, отозвать, лицензия, осу..."
1,30 января 2020 года,\nБанк России принял решение начиная с 2022 го...,"[банк, россия, принять, решение, начинать, исп..."
2,30 января 2020 года,\n\nВышел «Вестник Банка России» № 11–12 (2147...,"[выйти, вестник, банка, россия, январь, номер,..."
3,29 января 2020 года,\n\nПриказами Банка России от 05.09.2019 № ОД-...,"[приказ, банка, россия, общество, отозвать, ли..."
4,29 января 2020 года,\n\nВышел «Вестник Банка России» № 10 (2146) о...,"[выйти, вестник, банка, россия, январь, рубрик..."
5,29 января 2020 года,"\nБанк России принял решение о реализации мер,...","[банк, россия, принять, решение, реализация, м..."
6,29 января 2020 года,\n\nВ соответствии с решением Совета директоро...,"[соответствие, решение, совет, директор, банка..."
7,28 января 2020 года,\n\nБанк России пригласил инвестиционных консу...,"[банк, россия, пригласить, инвестиционный, кон..."
8,27 января 2020 года,\n\nВышел «Вестник Банка России» № 9 (2145) от...,"[выйти, вестник, банка, россия, январь, рубрик..."
9,24 января 2020 года,\n\nВышел «Вестник Банка России» № 8 (2144) от...,"[выйти, вестник, банка, россия, январь, номер,..."


In [160]:
tags_column = []

for word_list in df2.words:
    print(word_list)
    tags = []
    for word in word_list:
        if word in dict_terms:
            tags.append(emo_dict[emo_dict.term == word].tag.tolist()[0])
    if tags.count('PSTV') > tags.count('NGTV'):
        tags_column.append(1)
    if tags.count('NGTV') > tags.count('PSTV'):
        tags_column.append(-1)

['банк', 'россия', 'приказ', 'отозвать', 'лицензия', 'осуществление', 'банковский', 'операция', 'аграрный', 'профсоюзный', 'акционерный', 'коммерческий', 'банка', 'акционерный', 'общество', 'рег', 'москва', 'величина', 'актив', 'кредитный', 'организация', 'занимать', 'место', 'банковский', 'система', 'российский', 'банк', 'являться', 'участник', 'система', 'страхование', 'вклад', 'банк', 'россия', 'принять', 'решение', 'соответствие', 'плата', 'часть', 'ст', 'федеральный', 'закон', 'банка', 'банковский', 'руководствоваться', 'занижать', 'величина', 'необходимый', 'формирование', 'резерв', 'завышать', 'стоимость', 'имущество', 'цель', 'улучшение', 'финансовый', 'показатель', 'сокрытие', 'свой', 'реальный', 'финансовый', 'положение', 'отражение', 'финансовый', 'отчётность', 'принимать', 'кредитный', 'риск', 'реальный', 'стоимость', 'учесть', 'баланс', 'недвижимость', 'требование', 'банка', 'россия', 'выявить', 'снижение', 'капитал', 'кредитный', 'организация', 'являться', 'реальный', 'уг

['банк', 'россия', 'принять', 'решение', 'реализация', 'мера', 'направить', 'повышение', 'финансовый', 'устойчивость', 'севастопольский', 'морской', 'банк', 'севастополь', 'банк', 'комитет', 'банковский', 'надзор', 'банка', 'россия', 'соответствие', 'статья', 'федеральный', 'закон', 'несостоятельность', 'банкротство', 'утвердить', 'план', 'участие', 'государственный', 'корпорация', 'агентство', 'страхование', 'вклад', 'агентство', 'осуществление', 'мера', 'предупреждение', 'банкротство', 'банка', 'поддержка', 'деятельность', 'банка', 'повышение', 'финансовый', 'устойчивость', 'банк', 'россия', 'совместно', 'агентство', 'качество', 'инвестор', 'привлечь', 'банк', 'обеспечение', 'непрерывность', 'деятельность', 'банка', 'период', 'передача', 'банка', 'инвестор', 'приказ', 'банка', 'россия', 'соответствие', 'статья', 'федеральный', 'закон', 'несостоятельность', 'банкротство', 'назначить', 'временной', 'администрация', 'управление', 'севастопольский', 'морской', 'банк', 'лицо', 'агентство'

['цель', 'совершенствование', 'информационный', 'обеспечение', 'пользователь', 'рамка', 'участие', 'банка', 'россия', 'информационный', 'обеспечение', 'мониторинг', 'достижение', 'национальный', 'цель', 'развитие', 'российский', 'федерация', 'банк', 'россия', 'расширять', 'представление', 'информация', 'статистика', 'банковский', 'кредитование', 'юридический', 'лицо', 'индивидуальный', 'предприниматель', 'размещать', 'сайт', 'сведение', 'количество', 'заёмщик', 'предоставить', 'кредит', 'число', 'субъект', 'малое', 'среднее', 'предпринимательство', 'российский', 'федерация', 'публикация', 'информация', 'количество', 'заёмщик', 'предоставить', 'кредит', 'информация', 'количество', 'заёмщик', 'субъект', 'предоставить', 'имя', 'кредит', 'отразить', 'дать', 'количество', 'заёмщик', 'иметь', 'ссудный', 'задолженность', 'число', 'просрочить', 'количество', 'заёмщик', 'предоставить', 'кредит', 'течение', 'отчётный', 'период', 'привести', 'общий', 'количество', 'кредит', 'предоставить', 'кажды

['выйти', 'вестник', 'банка', 'россия', 'январь', 'рубрика', 'кредитный', 'организация', 'опубликовать', 'приказ', 'банка', 'россия', 'отзыв', 'лицензия', 'осуществление', 'банковский', 'операция', 'аннулирование', 'лицензия', 'осуществление', 'профессиональный', 'деятельность', 'рынок', 'ценный', 'бумага', 'кредитный', 'организация', 'нэклисбанк', 'москва;', 'назначение', 'временной', 'администрация', 'управление', 'кредитный', 'организация', 'нэклисбанк', 'москва', 'связь', 'отзыв', 'лицензия', 'осуществление', 'банковский', 'операций;', 'внесение', 'изменение', 'приложение', 'приказ', 'банка', 'россия', 'декабрь', 'внесение', 'изменение', 'приложение', 'приказ', 'банка', 'россия', 'ноябрь', 'разместить', 'сообщение', 'автономный', 'некоммерческий', 'организация', 'фонд', 'защита', 'вкладчик', 'приобретение', 'право', 'требование', 'часть', 'превышать', 'рубль', 'осуществление', 'дополнительный', 'компенсационный', 'выплата', 'вкладчик', 'средство', 'семнадцать', 'имущественный', 'вз

In [159]:
len(tags_column)

19